### Intervals analysis with fixed on-time
Since we noticed a drecreasing trend characterizing $\sigma_\mathrm{tot}$ as a function of $t_\mathrm{off}$, and since $f_\mathrm{sky}(t_\mathrm{off})$ presents an increasing trend, we hypothesized that these trends are a consequence of the dependence of $\mu$ (i.e., the mean value used to evaluate the standard deviations) from $t_\mathrm{off}$. To verify this assumption, we repeat the calculation of $\sigma_\mathrm{tot}$ performed in the intervals analysis, with a value of $\mu$ constant as a function of $t_\mathrm{off}$. To do so, we fix the number of periods (each equal to $t_\mathrm{on}+t_\mathrm{off}$) instead of the total duration of the simulation.

In [ ]:
using Stripeline    #to simulate the movement of the telescope
using Healpix    #to produce maps from the simulations
using Plots    #to produce graphs resulting from the analysis
using Statistics    #to evaluate the standard deviation easily
using LaTeXStrings    #to use the LaTex typography in the plots

Since the Julia language is optimized for the use of functions, we define a series of functions usefull in the main code of this program. The meaning of some of these, can be better understood by looking at the main cycle of the program written below.

In [ ]:
#This function is used to run the first simulation in which we are not interested in the operation of the calibrator,
#nor we need to know the hit count of every pixel.
function strip_map(time_range, map)
    dirs, _=genpointings(telescope_motors, Float64[0, 0, 1], time_samples_1)   
    #we only need the direction so we omit the second return value
    sp=0    #this counts the number of pixels seen by the telescope
    for k in 1:length(time_samples_1)
        colat, long=dirs[k, :]    #extract the colatitude and the longitude from dirs
        pixel_index=ang2pix(map, colat, long)
        map[pixel_index]=1    #we are not interested in understanding the pixels covered at least once, 
        #we do not need to know how many times the telescope covers every pixel
    end
    for i in 1:length(mapp.pixels)
        if mapp[i]>=1
            sp+=1
        end
        i+=1
    end
    return sp
end

#This function fill a single period (one t_off and one t_on) 
#of the tod representing the operation of the calibrator.
function period_tod(zeros, counts, durations, n, sampling)
    m=0
    while m<durations[2]    #m is a minute of the period, i is the index governing the tod
        if m<durations[1]    #we are in the on part of the period 
            #(we do not distinguish t_stab from t_cal in this case)
            v=1
            c=1
        end
        if m>=durations[1] && m<durations[2]    
            #we are in the off part of the period
            v=0
            c=2
        end
        b=((n*durations[2])+m)*sampling
        for a=1:sampling    
            #A complete cycle corresponds to an entire minute of the period,
            #which corresponds to a number=sampling of components of the tod.
            if (b+a)<=length(zeros)
                zeros[b+a]=v
                counts[c]+=1
                a+=1
            end
            if (b+a)>length(zeros)
                a=sampling+10
                m=durations[2]+10
            end
        end
        m+=1
    end
end

#This function fill the entire tod using period_tod several times.
function fill_tod(zeros, counts, durations, periods, sampling, y) 
    #For each moment in wich the calibrator is on, the tod value should be 1 instead of 0, 
    #at the beginning of the experiment the calibrator is on. 
    #During the first minute of each ignition the tod value will be 0.5 
    #so that we can distinguish it from the following nine.
    for n=0:periods    
        #we fill each period separately using the previous function
        period_tod(zeros, counts, durations, n, sampling)
        n+=1
    end
end

#This function produce a map based on the simulation of the movement of the telescope
#considering also the functioning of the calibrator (represented by the tod).
function project_to_map(time_samples, map, tod)
    telescope_motors(time_s)=(0.0, deg2rad(20.0), timetorotang(time_s, 1))
    dirs, _=genpointings(telescope_motors, Float64[0, 0, 1], time_samples)   
    #we only need the direction so we omit the second return value
    for k in 1:length(time_samples)
        colat, long=dirs[k, :]    
        #we extract the colatitude and the longitude from dirs
        pixel_index=ang2pix(map, colat, long)
        if tod[k]>0    
            #We are studying the situation from the observations point of view,
            #so we are considering t_on without make a distinction between t_cal and t_stab.
            map[pixel_index]+=1    
            #Every time a pixel is covered by the calibrating signal (stable or not) 
            #its colour value (hit count) is increased by a constant value. 
            #This way we can see both wich parts of the sky are covered and 
            #how many times the calibrator obscures them.  
        end
        if tod[k]==0 && map[pixel_index]==0
            map[pixel_index]=0.5   
            #this way we can distinguish the pixels in the strip visible to the telescope 
            #from the ones out of it
        end
        if map[pixel_index]>0.5 && (map[pixel_index]%1)!=0
            map[pixel_index]-=0.5   
            #We want the pixel_index to represent the number of passages of the calibrator
            #over a pixel, so we do not want our method to identify the strip to ruin 
            #the physical meaning of the hit count.
        end
    end
end

#This function fills the arrays keeping track of the hit count of the pixels under consideration.
function coveradge_counter(map, pixels, hit_count, tot_count_diff, on_count_diff, ideals) 
    j=1    
    #we need an index to controll the array that we are filling
    for r in 1:length(pixels)
        if map[r]==0.5    
            #this pixel was not covered by the calibrating signal so 
            #we put it in the strip pixels array but with its real count null
            hit_count[j]=0
            tot_count_diff[j]=-ideals
            #count_diff arrays are usefull in the standard deviation evaluation
            #ideals value is defined below
            j+=1    
            #we increment this index only when the hit count is >0
        end
        if map[r]>=1
            hit_count[j]=map[r]
            tot_count_diff[j]=hit_count[j]-ideals
            j+=1
        end
        r+=1
    end
end


Here we define a series of usefull variables that will be used in the main part of the code.

In [ ]:
ni=24    #number of ignitions (fixed)
cal_duration=10    #total ignition time (both stable and unstable) expressed in minutes
frequency=50    #sampling frequency in Hz (i.e., number of samples per seconds)
sampling=frequency*60    #samples per minute (i.e., sampling frequency * 60 seconds)
#We run a first simulation without the calibrator to verify the minimum time needed 
#by the telescope to cover completely the sky strip.
time_samples_1=0.0:1/frequency:(60*24*60)    #total time set to 24 hours
telescope_motors(time_s)=(0.0, deg2rad(20.0), timetorotang(time_s, 1))
mapp=Healpix.HealpixMap{Float64, RingOrder}(256)   #a map wich represents the whole sky
strip_pixels=strip_map(time_samples_1, mapp)    #here we run the simulation
on_samples=ni*cal_duration*frequency*60    #total time, expressed in seconds sampled at 50 Hz, during which 
#the calibrator is on (it is fixed!)
ideal_count=on_samples/strip_pixels    #If all the pixels were seen the same number of times, this would be their hit count.
cal_intervals=50:1:190
#We define an array of values that we will use as time between to ignitions to compare them, 
#here we express them in minutes.
#We define the associated array of total standard deviation
dev_sts_tot=zeros(Float64, length(cal_intervals), 1)
#the associated mean value will be defined using the number of pixels of the entire strip

Now we run a $\it{for}$ cycle in which, for each calibration strategy, we simulate the movement of the telescope and the action of the calibrator, then we evaluate the three observables of our analysis.

In [ ]:
for ind in 1:length(cal_intervals)    #we run the cycle for each calibration strategy previously defined
    obs_time=ni*(cal_intervals[ind]+cal_duration)    #total observational time in minutes (not fixed before)
    time_samples=0.0:(1/frequency):(obs_time*60)
    #array of time values in which we take measurements
    #i.e., time range in seconds sampled using the chosen frequency
    counts=[0, 0]   #Once filled, this array will be defined as follows:
    #first component = number of samples during which the calibrator is on, unstable or not; 
    #second component = number of samples during which the calibrator is off.
    durations=[cal_duration, cal_intervals[ind]+cal_duration]
    #This array represents the division of a period of the calibrator action expressed in minutes. 
    #In particular, each one of its components is the time at which a phase of the period ends (t_on, t_on+t_off).
    tod=zeros(Float64, length(time_samples), 1)
    #the tod wich will be used to simulate the calibrator, initialized to zero as a start 
    n_per=cld(obs_time, durations[2])    
    #number of periods in which the total time can be divided rounded to the maximum integer possible  
    fill_tod(tod, counts, durations, n_per, sampling, 0)    #we fill the tod using the functions previously defined  
    map=Healpix.HealpixMap{Float64, RingOrder}(256)   #now we create a map wich represents the whole sky
    project_to_map(time_samples, map, tod)    #here we run the simulation
    #We evaluate in a quantitative way the uniformity of the calibrating signal coverage by studying map.pixels 
    #i.e., the array containing the count of the calibrator coverages af each pixel.
    hit_count=zeros(Float64, strip_pixels, 1)    #hit count of every pixel in the strip
    tot_count_diff=zeros(Float64, strip_pixels, 1)    
    #difference between the real count of every pixel in the strip and the ideal value
    coveradge_counter(map, map.pixels, hit_count, tot_count_diff, ideal_count)   
    #we fill the arrays using the function previuosly created 
    #we adjourn the final array
    dev_sts_tot[ind]=stdm(tot_count_diff, ideal_count, corrected=true)    
    #standard deviation considering the whole strip (tot_dev_st)
    println("Component ", ind, " of ", length(cal_intervals), " has been processed.")
    #since the whole cycle could take a wile to be completed, we print a message to take track of the progress
    ind+=1    #we increase the index controlling the main cycle
end

The main output of the thesis are some plots usefull to analyse the results. Here we produce the plot resulting from this analysis.

In [ ]:
theme(:default)
plot(cal_intervals, dev_sts_tot, xlabel=L"t_\mathrm{off}\;[\mathrm{min}]", ylabel=L"\sigma_\mathrm{tot}", 
    minorgrid=true, markershape=:circle, markersize=2, linewidth=0.8, linealpha=0.6, legend=:false)
savefig("plot_sdt_ver2.pdf")